In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from utils.reparam_module import ReparamModule

In [16]:
data = torch.load('./pattern-toys.pth')
print(len(data))

11268


In [17]:
# Example to show how to find frequent sequential patterns
# from a given sequence database subject to constraints
from sequential.seq2pat import Seq2Pat, Attribute

# data = [
#     [1,2,112,2223,3,134],
#     [1,2,6456,324,1521345,123],
#     [1,2,125,4365,23421,203985],
#     [2,3,12358,4532,23541,38954],
#     [2,3,353,4234,45453,4678],
#     [2,3,234,3342,11234,465],
# ]

# Seq2Pat over 3 sequences
seq2pat = Seq2Pat(sequences=data, n_jobs=2, max_span=10)
# Patterns that occur at least twice (A-D)
patterns = seq2pat.get_patterns(min_frequency=2)
patterns_value = [_[:-1] for _ in patterns]
patterns_count = [_[-1] for _ in patterns]
# print(patterns)
print(len(patterns))

23426


In [18]:
patterns

[[4201, 4565, 52],
 [4565, 5799, 37],
 [5980, 6151, 37],
 [4201, 5799, 36],
 [4201, 4565, 5799, 28],
 [5980, 6151, 7245, 22],
 [5980, 7245, 22],
 [6151, 7245, 22],
 [5994, 5995, 21],
 [725, 1439, 19],
 [5439, 5659, 16],
 [5659, 6885, 16],
 [5991, 6214, 16],
 [1999, 1849, 15],
 [4201, 6333, 15],
 [4370, 5439, 15],
 [6673, 6885, 15],
 [18, 1445, 14],
 [5658, 5575, 14],
 [5799, 6333, 14],
 [6135, 6136, 14],
 [6233, 6692, 14],
 [244, 2739, 13],
 [5709, 5734, 13],
 [6182, 6692, 13],
 [6233, 6502, 13],
 [244, 1544, 12],
 [565, 675, 12],
 [4258, 5970, 12],
 [4289, 5172, 12],
 [4565, 6333, 12],
 [5659, 6673, 12],
 [5692, 5693, 12],
 [6692, 6700, 12],
 [1844, 1899, 11],
 [2256, 2636, 11],
 [4041, 5270, 11],
 [4933, 5312, 11],
 [5003, 2347, 11],
 [5247, 6685, 11],
 [5980, 6151, 7246, 11],
 [5980, 7246, 11],
 [6134, 6136, 11],
 [6151, 7246, 11],
 [6182, 6233, 11],
 [6233, 6182, 11],
 [6671, 6673, 11],
 [6788, 6673, 11],
 [6824, 6885, 11],
 [6885, 7236, 11],
 [72, 73, 10],
 [203, 1482, 10],
 [376,

In [19]:
original_train = torch.load('./dataset/amazon-toys-seq/toy/train_ori.pth')
len(original_train)

61616

In [20]:
max_seq_len = 20
def truncate_or_pad(seq):
    cur_seq_len = len(seq)
    if cur_seq_len > max_seq_len:
        return seq[-max_seq_len:]
    else:
        return seq + [0] * (max_seq_len - cur_seq_len)


train_set = set()
for pattern in patterns:
    seq = pattern[:-1]
    seq_len = len(seq)
    for _ in range(1, seq_len):
        train_set.add(tuple(
            truncate_or_pad(seq[:_]) + [seq[_]],
        ))

In [21]:
train_list = []
for _ in train_set:
    train_list.append([
        0,
        list(_)[:-1],
        list(_)[-1],
        sum([a != 0 for a in list(_)[:-1]]),
        1,
        0
    ])

In [22]:
len(train_list)

23426

In [23]:
len(original_train)

61616

In [24]:
torch.save(train_list + original_train, 'train_new.pth')

In [7]:
torch.save(train_list, 'train_new2-10-pure.pth')